sqlite

In [2]:
strs = "['學院基本介紹', '學院地址', '切換模型', '當前用量']"

In [7]:
type(eval(strs))

list

In [6]:
# SQL code to create the tables
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_line_id TEXT,
    user_name TEXT,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

create_user_permissions_table = """
CREATE TABLE IF NOT EXISTS user_permissions (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    collection_id INTEGER,
    is_permission BOOLEAN,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES users(id),
    FOREIGN KEY (collection_id) REFERENCES collections(id)
);
"""
create_collection_table = """
CREATE TABLE IF NOT EXISTS collections (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    collection_name TEXT NOT NULL,
    prompt TEXT,
    files TEXT,
    new_files TEXT,
    is_update boolean DEFAULT 0,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

# List of SQL commands to create indexes
index_commands = [
    "CREATE INDEX idx_user_line_id ON Users(id);",
    "CREATE INDEX idx_user_id ON UserPermissions(id);"
]


import sqlite3
import json
def get_connection():
    conn = sqlite3.connect('../../database.db')
    return conn

def create_table_collections():
    conn = get_connection()
    cursor = conn.cursor()
    # Create new table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS collections (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            collection_name TEXT NOT NULL,
            prompt TEXT,
            files TEXT,
            new_files TEXT,
            is_update boolean DEFAULT 0,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    conn.commit()
    conn.close()

def get_collections():
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute('SELECT * FROM collections')

    rows = cursor.fetchall()
    conn.close()

    # Get column names from cursor description
    column_names = [column[0] for column in cursor.description]

    # Convert rows to dictionaries
    dict_rows = [dict(zip(column_names, row)) for row in rows]

    # Convert 'files' from str to list
    for row in dict_rows:
        row['files'] = json.loads(row['files'])
        row['new_files'] = json.loads(row['new_files'])
        
    return dict_rows

# insert a new row into the collections table
def insert_collection(collection_name, prompt=None, files='[]', new_files='[]'):
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute('''
        INSERT INTO collections (collection_name, prompt, files, new_files)
        VALUES (?, ?, ?, ?)
    ''', (collection_name, prompt, files, new_files))

    conn.commit()
    conn.close()

def update_collection(id, collection_name=None, prompt=None, files=None, new_files=None):
    conn = get_connection()
    cursor = conn.cursor()

    # 使用字典來儲存要更新的欄位和值
    updates = {
        'collection_name': collection_name,
        'prompt': prompt,
        'files': files,
        'new_files': new_files
    }

    # 過濾掉值為 None 的項目
    updates = {column: value for column, value in updates.items() if value is not None}

    # 如果有要更新的欄位，則生成 SQL 語句並執行
    if updates:
        set_clause = ', '.join([f"{column} = ?" for column in updates.keys()])
        parameters = list(updates.values())
        parameters.append(id)

        sql = f"UPDATE collections SET {set_clause}, is_update = 1, updated_at = CURRENT_TIMESTAMP WHERE id = ?"
        cursor.execute(sql, parameters)
        conn.commit()

    conn.close()

# delete the collections table
def delete_table_collections():
    conn = get_connection()
    cursor = conn.cursor()
    # Delete table
    cursor.execute('''
        DROP TABLE IF EXISTS collections
    ''')
    conn.commit()
    conn.close()

# create a new table
def create_table(sql):
    conn = get_connection()
    cursor = conn.cursor()
    # Create new table
    cursor.execute(sql)
    conn.commit()
    conn.close()


def insert_user(user_line_id, user_name):
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute('''
        INSERT INTO users (user_line_id, user_name)
        VALUES (?, ?)
    ''', (user_line_id, user_name))

    conn.commit()
    conn.close()

def get_users():
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute('SELECT * FROM users')

    rows = cursor.fetchall()
    conn.close()

    # Get column names from cursor description
    column_names = [column[0] for column in cursor.description]

    # Convert rows to dictionaries
    dict_rows = [dict(zip(column_names, row)) for row in rows]

    return dict_rows

def delete_user_permission(user_id, collection_id):
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute('''
        DELETE FROM user_permissions
        WHERE user_id = ? AND collection_id = ?
    ''', (user_id, collection_id))

    conn.commit()
    conn.close()

def get_user_permissions():
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute('SELECT * FROM user_permissions')

    rows = cursor.fetchall()
    conn.close()
    
    # Get column names from cursor description
    column_names = [column[0] for column in cursor.description]

    # Convert rows to dictionaries
    dict_rows = [dict(zip(column_names, row)) for row in rows]


    return dict_rows

def create_table_model_status():
    conn = get_connection()
    cursor = conn.cursor()
    # Create new table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS model_status (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            status TEXT,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    conn.commit()
    conn.close()

def insert_model_status(status):
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute('''
        INSERT INTO model_status (status)
        VALUES (?)
    ''', (status,))

    conn.commit()
    conn.close()

def update_model_status(status):
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute('''
        UPDATE model_status
        SET status = ?, updated_at = CURRENT_TIMESTAMP
        WHERE id = 1
    ''', (status,))

    conn.commit()
    conn.close()

def get_model_status():
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute('''
        SELECT status, updated_at
        FROM model_status
        WHERE id = 1
    ''')

    result = cursor.fetchone()

    conn.close()

    return result



In [10]:
update_model_status('pending')

In [11]:
def update_model_status(status):
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute('''
        UPDATE model_status
        SET status = ?, updated_at = CURRENT_TIMESTAMP
        WHERE id = 1
    ''', (status,))

    conn.commit()
    conn.close()

get_model_status()

('pending', '2024-07-09 07:06:42')

In [7]:
def get_user_permissions_info():
    conn = get_connection()
    cursor = conn.cursor()


    # 獲取所有唯一的產品名稱
    cursor.execute("SELECT DISTINCT collection_name FROM collections")
    collection_names = [row[0] for row in cursor.fetchall()]
    # 構建動態 SQL 查詢
    columns = ", ".join([f"MAX(CASE WHEN c.collection_name = '{collection_name}' THEN up.is_permission ELSE 0 END) AS {collection_name}" for collection_name in collection_names])

    cursor.execute(f'''
    SELECT 
        u.id,
        u.user_name,
        {columns}
    FROM 
        users u
    LEFT JOIN 
        user_permissions up ON u.id = up.user_id
    LEFT JOIN 
        collections c ON up.collection_id = c.id
    GROUP BY 
        u.id, u.user_name;
    ''')

    rows = cursor.fetchall()
    conn.close()

    # Get column names from cursor description
    column_names = [column[0] for column in cursor.description]

    # Convert rows to dictionaries
    dict_rows = [dict(zip(column_names, row)) for row in rows]

    return dict_rows

get_user_permissions_info()

[{'id': 1, 'user_name': 'Michael', 'testing_num1': 1, 'testing_num2': 1},
 {'id': 2, 'user_name': 'Ivan', 'testing_num1': 1, 'testing_num2': 1},
 {'id': 3, 'user_name': 'Levi', 'testing_num1': 1, 'testing_num2': 1}]

In [13]:
def transfer_name2id(user_name, collection_name):
    conn = get_connection()
    cursor = conn.cursor()
    # 獲取所有唯一的產品名稱
    cursor.execute("SELECT id FROM users where user_name = ?", (user_name,))
    user_id = cursor.fetchall()[0][0]

    cursor.execute("SELECT id FROM collections where collection_name = ?", (collection_name,))
    collection_id = cursor.fetchall()[0][0]
    return user_id, collection_id

def insert_user_permission(user_id, collection_id):
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute('''
        with collection as (
            select id from collections where id = ?
        )
        INSERT INTO user_permissions (user_id, collection_id, is_permission)
        VALUES (?, ?, 1)
    ''', (user_id, collection_id))

    conn.commit()
    conn.close()

insert_user_permission("Levi", "testing_num2")

3 2


In [15]:
def transfer_name2id(user_name, collection_name):
    conn = get_connection()
    cursor = conn.cursor()
    # 獲取所有唯一的產品名稱
    cursor.execute("SELECT id FROM users where user_name = ?", (user_name,))
    user_id = cursor.fetchall()[0][0]

    cursor.execute("SELECT id FROM collections where collection_name = ?", (collection_name,))
    collection_id = cursor.fetchall()[0][0]
    return user_id, collection_id
transfer_name2id("Michael", "testing_num2")

(1, 2)

In [ ]:
def remove_user_permission(user_id, collection_id):
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute('''
        DELETE FROM user_permissions
        WHERE user_id = ? AND collection_id = ?
    ''', (user_id, collection_id))

    conn.commit()
    conn.close()

remove_user_permission(1,1)
remove_user_permission(2,1)
remove_user_permission(3,1)

In [ ]:
insert_user_permission(1, 1)

In [ ]:
get_user_permissions()

In [ ]:
create_table_model_status()
insert_model_status("pending")

In [ ]:
update_model_status("updating")

In [ ]:
get_model_status()

In [ ]:
def get_model_status():
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute('''
        SELECT status, updated_at
        FROM model_status
        WHERE id = 1
    ''')

    rows = cursor.fetchone()

    conn.close()
    return rows

get_model_status()

In [ ]:
update_model_status("completed")

In [ ]:
get_users()

In [ ]:
import pandas as pd
def read_sqlite_table(table_name=None, sql_query=None):
    """
    Reads data from a SQLite database into a pandas DataFrame.

    Parameters:
    - table_name (str, optional): Name of the table to read. Use only if sql_query is None.
    - sql_query (str, optional): Custom SQL query to execute. Overrides table_name if provided.

    Returns:
    - DataFrame containing the data retrieved from the database.
    """

    # Establish connection to the SQLite database
    conn = get_connection()
    
    # Determine the SQL query to execute
    if sql_query is None:
        if table_name is None:
            raise ValueError("Either table_name or sql_query must be provided")
        sql_query = f"SELECT * FROM {table_name}"

    # Load the data into a pandas DataFrame
    dataframe = pd.read_sql_query(sql_query, conn)

    # Close the database conn
    conn.close()

    return dataframe

In [ ]:
# sql_query = """SELECT * FROM users""" 
df = read_sqlite_table("users")

In [ ]:
df

In [ ]:
get_users()

In [ ]:
insert_user_permission(1, 2, 1)

In [ ]:
insert_user('U123456','Ivan')

In [ ]:
get_user_permissions()

In [ ]:
get_collections()

In [ ]:
type(get_collections()[0]["files"])

In [ ]:
insert_user("testing_lineid", "Michael")

In [ ]:
create_table(create_users_table)
create_table(create_user_permissions_table)

In [ ]:
def get_tables():
    conn = get_connection()
    cursor = conn.cursor()

    # Get all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    conn.close()

    # Extract table names from the result
    table_names = [table[0] for table in tables]

    return table_names

tables = get_tables()
print(tables)

gcp

In [ ]:
# Imports the Google Cloud client library
from google.cloud import storage
import os 

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.abspath('../utility-encoder-420001-0db7ee074ec6.json')
# Instantiates a client
storage_client = storage.Client()

In [ ]:
buckets = storage_client.list_buckets()

# for bucket in buckets:
#     print(bucket.name)

In [ ]:
buckets

In [ ]:
for bucket in buckets:
    print(bucket.name)

In [ ]:
from google.cloud import storage

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )

In [ ]:
bucket = storage_client.bucket('laoshifu')
blob = bucket.blob("Student Insurance Benefits.pdf")
blob.delete()

In [ ]:
storage_client = storage.Client()
bucket = storage_client.bucket('your-bucket-name')
blob = bucket.blob('your-file-name')
if blob.exists():
    blob.delete()
    print('File deleted successfully')
else:
    print('File not found in Google Cloud Storage')


In [ ]:
from google.cloud import storage
import io

# 初始化 GCP Storage 客戶端
client = storage.Client()
bucket = client.bucket('laoshifu')

# 從 Cloud Storage 下載 PDF 文件
blob = bucket.blob('Student Insurance Benefits.pdf')
pdf_bytes = blob.download_as_bytes()
pdf_file = io.BytesIO(pdf_bytes)

In [ ]:
pdf_file

In [ ]:
from google.cloud import storage
import fitz
import io

def process_pdf(bucket_name, source_blob_name):
    # Instantiates a client
    storage_client = storage.Client()

    # Get GCS bucket
    bucket = storage_client.bucket(bucket_name)

    # Get the blob
    blob = bucket.blob(source_blob_name)

    # Download the contents of the blob as a string
    pdf_data = blob.download_as_bytes()

    # Open the PDF file
    doc = fitz.open("pdf", pdf_data)
    text = ""
    for page in doc:
        text += page.get_text()

    if text:
        return text
    return None

In [ ]:
text = process_pdf('laoshifu', '1/Michael Chen Resume.pdf')

In [ ]:
text

In [28]:
import sqlite3
import json
import pandas as pd

def get_connection():
    conn = sqlite3.connect('../../database.db')
    return conn

def create_table_collections():
    conn = get_connection()
    cursor = conn.cursor()
    # Create new table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS collections (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            collection_name TEXT NOT NULL,
            new_collection_name TEXT,
            prompt TEXT,
            files TEXT,
            new_files TEXT,
            is_update boolean DEFAULT 0,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    conn.commit()
    conn.close()

# insert a new row into the collections table
def insert_collection(collection_name, prompt=None, files='[]', new_files='[]'):
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute('''
        INSERT INTO collections (collection_name, prompt, files, new_files)
        VALUES (?, ?, ?, ?)
    ''', (collection_name, prompt, files, new_files))

    conn.commit()
    conn.close()

def update_collection(id, collection_name=None, prompt=None, files=None, new_files=None):
    conn = get_connection()
    cursor = conn.cursor()

    # 使用字典來儲存要更新的欄位和值
    updates = {
        'new_collection_name': collection_name,
        'prompt': prompt,
        'files': files,
        'new_files': new_files
    }

    # 過濾掉值為 None 的項目
    updates = {column: value for column, value in updates.items() if value is not None}

    # 如果有要更新的欄位，則生成 SQL 語句並執行
    if updates:
        set_clause = ', '.join([f"{column} = ?" for column in updates.keys()])
        parameters = list(updates.values())
        parameters.append(id)

        sql = f"UPDATE collections SET {set_clause}, is_update = 1, updated_at = CURRENT_TIMESTAMP WHERE id = ?"
        cursor.execute(sql, parameters)
        conn.commit()

    conn.close()
    
# delete the collections table
def delete_table_collections():
    conn = get_connection()
    cursor = conn.cursor()
    # Delete table
    cursor.execute('''
        DROP TABLE IF EXISTS collections
    ''')
    conn.commit()
    conn.close()

def get_collections():
    conn = get_connection()
    cursor = conn.cursor()

    cursor.execute('''
        SELECT
            id,
            CASE
                WHEN is_update=1 THEN new_collection_name
            ELSE collection_name
            END AS collection_name,
            prompt,
            new_files,
            files,
            is_update
        FROM collections''')

    rows = cursor.fetchall()
    conn.close()

    # Get column names from cursor description
    column_names = [column[0] for column in cursor.description]

    # Convert rows to dictionaries
    dict_rows = [dict(zip(column_names, row)) for row in rows]

    # Convert 'files' from str to list
    for row in dict_rows:
        row['files'] = json.loads(row['files'])
        row['new_files'] = json.loads(row['new_files'])
        
    return dict_rows

In [31]:
create_table_collections()

In [32]:
insert_collection('Group1', 'Group1 提示詞', '[]', '[]')

In [33]:
insert_collection('Group2', 'Group2 提示詞', '[]', '[]')

In [34]:
insert_collection('Group3', 'Group3 提示詞', '[]', '[]')

In [36]:
update_collection(2, prompt="Group2 系統提示詞")

In [84]:
get_collections()

[{'id': 1,
  'collection_name': '',
  'prompt': 'Group1 系統提示詞',
  'new_files': ['Nonimmigrant Visa - Confirmation Page.pdf'],
  'files': ['疫苗証明.pdf', 'Nonimmigrant Visa - Confirmation Page.pdf'],
  'is_update': 1},
 {'id': 2,
  'collection_name': 'Group2',
  'prompt': 'Group2 系統提示詞',
  'new_files': [],
  'files': [],
  'is_update': 0},
 {'id': 3,
  'collection_name': 'Group3',
  'prompt': 'Group3 提示詞',
  'new_files': [],
  'files': [],
  'is_update': 0}]

In [85]:
class SqliteManager:
    def __init__(self):
        self.conn = sqlite3.connect('../../database.db')
    def reset_udpate_status(self):
        cursor = self.conn.cursor()
        cursor.execute("""
            UPDATE collections 
            SET 
                new_files = '[]', 
                is_update = 0,
                collection_name = CASE 
                    WHEN new_collection_name <> '' THEN new_collection_name 
                    ELSE collection_name 
                END,
                new_collection_name = ''
        """)
        self.conn.commit()
        self.conn.close()

sql = SqliteManager()
sql.reset_udpate_status()